In [ ]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

In [ ]:
encoding_dim = 3

In [ ]:
input_img = Input(shape=(784,),name="input_layer")
encoded = Dense(encoding_dim, activation='relu', name="encoder_layer")(input_img)
decoded = Dense(784, activation='sigmoid', name="decoder_layer")(encoded)

In [ ]:
# end-to-end reconstruction
autoencoder = Model(input_img, decoded) 

# map input to encoded representation
encoder = Model(input_img, encoded) 

# decode a representation
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
print(autoencoder.summary())
SVG(model_to_dot(autoencoder).create(prog='dot', format='svg'))

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=1024,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
encoded_imgs

In [ ]:
df = pd.DataFrame(data=encoded_imgs)
df['label'] = y_test

plot_3d = plt.figure(figsize=(18, 18)).gca(projection='3d')
plot_3d.view_init(20, 35)
plot_3d.scatter(df[0], df[1], df[2], c=df.label, cmap='rainbow' )
plt.show()

In [ ]:
import matplotlib.pyplot as plt

n = 10 
plt.figure(figsize=(20, 4))
for i in range(n):
    # original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()